<h1 style="text-align: center;"> Final Project </h1>

<h2 style="text-align: center;">  Pandemic Simulator </h2>
<h3 style="text-align: center;">Team members: Matt Mitchell, Nick Vino, and Logan Jolicoeur </h3>

# Section I
## Introduction
In this notebook, we'll develop a model of a pandemic as it directly spreads in a susceptible population, and explore the effectiveness of interventions such as vaccination and quarantining . Epidemiologic modeling is important for understanding disease spread and developing plans to control the spread. In Section II, we establish a strong foundation by building the simpliest SI model to discover the underlying ordinary differential equations that dictate the rules for the system and an inuitive sense for how to extend the model to capture more nuances which correspond to a more realistic model.

According to 2022 census records, the city of Banner Elk in North Carolina has a population of approximately 1,000 people. Banner Elk will be the center of our attention as we simulate the spread of an infection in the city. To simplify our study we assume the town will have a fixed population, i.e., no vital processes. We introduce well-known models of infectious disease, the SI model, the SIR (Kermack-McKendrick) model, SIS model, SIRS model, SIRV model and use them to explain the progression of the disease over the course of 2,000 days.



# Section II
## Implementation
The following models were produced with the Epidemics on Networks (EoN) package for python

## Citations:
[Journal of Open Source Software publication](https://doi.org/10.21105/joss.01731)

[EoN documentation](https://epidemicsonnetworks.readthedocs.io/en/latest/EoN.html)

The following packages are necessary for this notebook:

In [ ]:
import EoN
import networkx as nx
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from PIL import Image
from IPython.display import HTML

## Node Implementation G(n,p)

Each person in these models are connected to each other via edges. Edges are a representation of how they different nodes(persons) know or interact with one another. 

![Alt text](node_figure.png)

If an $I(t)$ node is connected to a $S(t)$ this does not mean the $S(t)$ node will become infected. There are rules in place that dictate this. For example, a $I(t)$ node has a choice on whether to "go out" or quarantine. If they decide to go out, then they risk infecting on of their friends and family. Further, each node, when infected has a $infection.prob$, if the $infection.prob$ is greater than a random generated number, they will infect that node. It is important to note that on average, each node has one to five connections. This can increase dramatically, if a node decides to go to a party or a big event, with other nodes. To model this, we implemented an increase of connections between nodes, based on friends/families in the community.

![Alt text](../../../../../C:/Users/warso/Desktop/final_project_407/increase_nodeage.png)


## SI Model

Initially, we consider a model with no recovery rate, a person is considered to be either susceptible or infected. A real life example of a SI disease is the herpesvirus family of viruses which characteristically establishes latent infection within specific tissues [[1]](https://www.ncbi.nlm.nih.gov/books/NBK8157/). The interaction between the two classes of population results in a flow from susceptible to infected over time. We can write this as the following rates of change for S and I respectively:

$\frac{dS}{dt} = -rSI$

$\frac{dI}{dt} = rSI$

This nonlinear system of differential equation can be solved assuming the population remains constant by adding the two equations. We arrive at $\frac{dS}{dt} + \frac{dI}{dt} = 0$ which can be integrated giving us $S(t) + I(t) = S(0) + I(0)$. The initial population is $N$ so we write $S(t) + I(t) = N$ to allow us to replace $S$ in the rates above with $N - I$. Our new equation is as follows

$\frac{dI}{dt} = r(N-I)I$.

If $I(0) > 0$, then the equilibrium $N$ will be approached as $t$ approaches infinity. Simply put the SI model predicts that everyone will become infected as long as there is an initial infected population.

Furthermore, we choose to investigate the value $R_0$ which is defined as the total number of new infected caused by each infected individual, on average.

$R_0 = 0.0015$

For a population of 1000 we expect an infected person to cause 15 new infections on average.

We can set parameters of the model as follows:

In [ ]:
N=1000
rho = 0.01      #intial infected
kave = 5
gamma = 0       #recovery rate
tau = 0.0015    #transmision rate

S0 = (1-rho)*N
I0 = rho*N
SI0 = (1-rho)*kave*rho*N
SS0 = (1-rho)*kave*(1-rho)*N
t, S, I = EoN.SIS_homogeneous_pairwise(S0, I0, SI0, SS0, kave, tau, gamma, tmax=2000)

plt.plot(t, S, label = 'S')
plt.plot(t, I, label = 'I')
plt.xlabel('$t$')
plt.ylabel('Population')
plt.legend()
plt.show()

## SIR Model

The SIR model is similar to the SIS model, but in this model after susceptible become infected they become R(t) or recovered. Where they cannot become infected again. 

The susceptible equation is as follows: 

$\frac{dS}{dt} = $-b * $s(t) * I(t)$

The recovered equation is:

$\frac{dr}{dt} = k * i(t)$

The infected equation is:

$\frac{dS}{dt} + \frac{di}{dt} + \frac{dr}{dt} = 0$

These equations are utilized to develop our models below, we some variation.

In [ ]:
N = 1000            #population
rho = 0.01          #initial infected
kave = 5            #average degree
gamma = 0.0045      #recovery rate
tau = 0.007         #transmission rate

phiS0 = 1-rho
def psi(x):
    return (1-rho)* np.exp(-kave*(1-x))
def psiPrime(x):
    return (1-rho)*kave*np.exp(-kave*(1-x))

t, S, I, R = EoN.EBCM(N, psi, psiPrime, tau, gamma, phiS0, tmax = 2000)
plt.plot(t, S, label = 'S')
plt.plot(t, I, label = 'I')
plt.plot(t, R, label = 'R')
plt.xlabel('$t$')
plt.ylabel('Population')
plt.legend()
plt.show()

## SIS Model

$R_0$ = 2 * 0.01 / 5 = 0.004

$R_0$ for a population of 1000 is 4. We can conclude that on average an infected node infects 4 new infections on average.

In [ ]:
N = 1000
rho = 0.01
kave = 5

gamma = 0.01
tau = 2*gamma/ kave

S0 = (1-rho)*N
I0 = rho*N
SI0 = (1-rho)*kave*rho*N
SS0 = (1-rho)*kave*(1-rho)*N
t, S, I = EoN.SIS_homogeneous_pairwise(S0, I0, SI0, SS0, kave, tau, gamma, tmax=2000)
plt.plot(t, S, label = 'S')
plt.plot(t, I, label = 'I')
plt.xlabel('$t$')
plt.ylabel('Predicted numbers')
plt.legend()
plt.show()

## SIRS Model

In [ ]:
N = 1000
kave = 5    #expected number of partners
G = nx.fast_gnp_random_graph(N, kave/(N-1))

H = nx.DiGraph()  #DiGraph showing possible transitions that don't require an interaction
H.add_edge('I', 'R', rate = 0.0025)   #Recovery rate
H.add_edge('R', 'S', rate = 0.001)   #Recovered -> Susceptible

J = nx.DiGraph()    #DiGraph showing transition that does require an interaction.
J.add_edge(('I', 'S'), ('I', 'I'), rate = 0.0065)  #IS->II

IC = defaultdict(lambda: 'S')
for node in range(10):
    IC[node] = 'I'

return_statuses = ('S', 'I', 'R')

t, S, I, R = EoN.Gillespie_simple_contagion(G, H, J, IC, return_statuses, tmax = 2000)

plt.plot(t, S, label = 'S')
plt.plot(t, I, label = 'I')
plt.plot(t, R, label = 'R')
plt.legend()

## SIRV Model

The SIRV model includes the vaccination status of the different people or nodes within the simulation. Each person has a chance to become vaccinated at the start, but they must be susceptible. This adds an interesting layer of whether or not vaccinations could assist with the recovery of a population. We also added in a decrease of variability of the vaccine. What this means that after a certain rate of time, the vaccination will begin to wear off and people can become sick again, if they have not yet recovered that is.


In [ ]:
G = nx.grid_2d_graph(32,32) #each node is (u,v) where 0<=u,v<=32    32x32 = 1024 population
#we'll initially infect those near the middle
initial_infections = [(u,v) for (u,v) in G if 14<u<18 and 14<v<18] # 3x3 = 9 initial infected
H = nx.DiGraph() #the spontaneous transitions
H.add_edge('Sus', 'Vac', rate = 0.0003)
H.add_edge('Inf', 'Rec', rate = 0.0025)
J = nx.DiGraph() #the induced transitions
J.add_edge(('Inf', 'Sus'), ('Inf', 'Inf'), rate = 0.01)

IC = defaultdict(lambda:'Sus')
for node in initial_infections:
    IC[node] = 'Inf'
    
return_statuses = ['Sus', 'Inf', 'Rec', 'Vac']

color_dict = {'Sus': '#E69A00','Inf':'#ff2000', 'Rec':'#CD9AB3','Vac': '#03254c'}
pos = {node:node for node in G}
tex = False
sim_kwargs = {'color_dict':color_dict, 'pos':pos, 'tex':tex}

sim = EoN.Gillespie_simple_contagion(G, H, J, IC, return_statuses, tmax=2000, return_full_data=True, sim_kwargs=sim_kwargs)

times, D = sim.summary()

newD = {'Sus+Vac':D['Sus']+D['Vac'], 'Inf+Rec' : D['Inf'] + D['Rec']}

new_timeseries = (times, newD)
sim.add_timeseries(new_timeseries, label = 'Simulation', color_dict={'Sus+Vac':'#E69A00', 'Inf+Rec':'#CD9AB3'})

sim.display(time=1, node_size = 4, ts_plots=[['Inf'], ['Sus+Vac', 'Inf+Rec']])

ani=sim.animate(ts_plots=[['Inf'], ['Sus+Vac', 'Inf+Rec']], node_size = 4)
ani.save('SIRV_animate.gif', writer="pillow", dpi=100)

Demonstration of the vaccine wear off.

In [ ]:
G = nx.grid_2d_graph(32,32) #each node is (u,v) where 0<=u,v<=32
#we'll initially infect those near the middle
initial_infections = [(u,v) for (u,v) in G if 14<u<18 and 14<v<18]
H = nx.DiGraph() #the spontaneous transitions
H.add_edge('Sus', 'Vac', rate = 0.0005)
H.add_edge('Inf', 'Rec', rate = 0.0025)
J = nx.DiGraph() #the induced transitions
J.add_edge(('Inf', 'Sus'), ('Inf', 'Inf'), rate = 0.01)

IC = defaultdict(lambda:'Sus')
for node in initial_infections:
    IC[node] = 'Inf'
    
return_statuses = ['Sus', 'Inf', 'Rec', 'Vac']

color_dict = {'Sus': '#E69A00','Inf':'#ff2000', 'Rec':'#CD9AB3','Vac': '#03254c'}
pos = {node:node for node in G}
tex = False
sim_kwargs = {'color_dict':color_dict, 'pos':pos, 'tex':tex}

sim = EoN.Gillespie_simple_contagion(G, H, J, IC, return_statuses, tmax=2000, return_full_data=True, sim_kwargs=sim_kwargs)

times, D = sim.summary()

newD = {'Sus+Vac':D['Sus']+D['Vac'], 'Inf+Rec' : D['Inf'] + D['Rec']}

new_timeseries = (times, newD)
sim.add_timeseries(new_timeseries, label = 'Simulation', color_dict={'Sus+Vac':'#E69A00', 'Inf+Rec':'#CD9AB3'})

sim.display(time=1, node_size = 4, ts_plots=[['Inf'], ['Sus+Vac', 'Inf+Rec']])

ani2=sim.animate(ts_plots=[['Inf'], ['Sus+Vac', 'Inf+Rec']], node_size = 4)
ani2.save('SIRV_animate.gif', writer="pillow", dpi=100)

In [ ]:

HTML(ani.to_jshtml())

In [ ]:
HTML(ani2.to_jshtml())

## Masks

In this scenario, we implement masks. We will be using the SIR model to demonstrate how effective masks can be in curtailing the pandemic. 

Based on mayoclinic.org report (https://www.mayoclinic.org/diseases-conditions/coronavirus/in-depth/coronavirus-mask/art-20485449). 

The report indicates that 83% of masks assist with curtailing COVID-19. This is not a representation of how COVID-19 affects the population, but it is a starting point for how effective masks can be in a "general" pandemic scenario. Without masks, we start an infection rate of 0.0065. Taking 83% of 0.0065, we get 0.001105. As such this is the representation of how masks can affect a pandemic, if the population were to wear them at time(1).


In [ ]:
N = 1000
kave = 5    #expected number of friends/family
G = nx.fast_gnp_random_graph(N, kave/(N-1))

H = nx.DiGraph()
H.add_edge('I', 'R', rate = 0.0025)   #Recovery rate
H.add_edge('R', 'S', rate = 0.001)   #Recovered -> Susceptible

J = nx.DiGraph()  
J.add_edge(('I', 'S'), ('I', 'I'), rate = 0.001105)  #IS->II 83% of 0.0065 (this is based on the stat that 83% of masks curtail people getting infect with COVID)

IC = defaultdict(lambda: 'S')
for node in range(10):
    IC[node] = 'I'

return_statuses = ('S', 'I', 'R')

t, S, I, R = EoN.Gillespie_simple_contagion(G, H, J, IC, return_statuses, tmax = 2000)

plt.plot(t, S, label = 'S')
plt.plot(t, I, label = 'I')
plt.plot(t, R, label = 'R')
plt.ylabel('Population')
plt.xlabel('$t$')
plt.legend()

## Without Masks

This scenario shows what happens if we do not have masks and the population does not wear them at all. The rate will be the full 0.0065 rate. Show casing that there is nothing stopping the infection from spreading betweens friends and family, except the chance of getting the virus.

In [ ]:
N = 1000
kave = 5    #expected number of friends/family
G = nx.fast_gnp_random_graph(N, kave/(N-1))

H = nx.DiGraph()
H.add_edge('I', 'R', rate = 0.0025)   #Recovery rate
H.add_edge('R', 'S', rate = 0.001)   #Recovered -> Susceptible

J = nx.DiGraph()  
J.add_edge(('I', 'S'), ('I', 'I'), rate = 0.0065)  #IS->II, half the rate of  demonstrating an half in infection occuring if masks are worn by the population

IC = defaultdict(lambda: 'S')
for node in range(10):
    IC[node] = 'I'

return_statuses = ('S', 'I', 'R')

t, S, I, R = EoN.Gillespie_simple_contagion(G, H, J, IC, return_statuses, tmax = 2000)

plt.plot(t, S, label = 'S')
plt.plot(t, I, label = 'I')
plt.plot(t, R, label = 'R')
plt.ylabel('Population')
plt.xlabel('$t$')
plt.legend()

As you can tell above. We have a significant increase of people that acquired the infection, than those that didn't. With a mask stopping 83% of an infection actually occuring for S(t) people, we had a significant decrease. Based on our evidence, masks do something and not anything at all. However, different types of masks and their grade is not included into these scenarios, so do keep that in mind.

## Section III

# Conclusion

To conclude, the different models above assist in a better understanding of how diseases can spread, and with differing variations we can better understand how lockdowns, social gatherings, quarantines, or masks can either improve or unimprove a pandemic. It is important to note that these models do not take into account all variables that can occur during a pandemic, because humans and viruses can be unpredictable sometimes. Which leaves much to understand. 